In [ ]:
from ultralytics import YOLO
import cv2
import os
import csv
from datetime import datetime

# Charger le modèle YOLOv11 (ou un modèle personnalisé)
model = YOLO("yolov11.pt")  # ou ton propre modèle : "runs/train/exp/weights/best.pt"

# Dossier pour sauvegarder les images
output_dir = "detections"
os.makedirs(output_dir, exist_ok=True)

# Fichier CSV pour stocker les comptages
csv_file = os.path.join(output_dir, "comptage_pneus.csv")

# Initialiser le CSV si besoin
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["datetime", "nb_pneus_detectés", "nom_image"])

# Ouvrir la caméra (0 = webcam intégrée)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Erreur : impossible d'ouvrir la caméra.")
    exit()

print("Appuyez sur Q pour quitter...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Détection avec YOLO
    results = model(frame)

    # Compter les pneus (si le modèle contient la classe "tire" ou "pneu")
    pneu_count = 0
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            label = model.names[cls]
            if "tire" in label.lower() or "pneu" in label.lower():
                pneu_count += 1

    # Afficher le nombre sur la vidéo
    cv2.putText(frame, f"Pneus détectés: {pneu_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Montrer le flux vidéo
    cv2.imshow("YOLOv11 - Comptage pneus", frame)

    # Sauvegarder image et comptage
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    img_name = f"{timestamp}.jpg"
    img_path = os.path.join(output_dir, img_name)
    cv2.imwrite(img_path, frame)

    with open(csv_file, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, pneu_count, img_name])

    # Quitter si Q est pressé
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
